In [9]:
# 크롤링 작업을 위한 라이브러리 임포트
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import requests
import pandas as pd
import json
import time   # 코드 진행 지연을 위한 time 임포트
from selenium.webdriver.common.by import By   # 2022년 7월 이후 selenium 업데이트로 인한 xpath 추적 임포트
from selenium.webdriver.common.keys import Keys

#########################################################
####################  API  입력  부분  ####################
#########################################################

##### 카카오 주소를 좌표(위도, 경도)로 변환 api
## 개인 api key를 할당 받아 적용
api_key = '054a38bc8bc9ee2f8fadaaad44e27f97'
def addr_to_lat_lon(addr, index):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={addr}'
    #print(url)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    print(index, url)

    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])


##### 카카오맵 minimap으로 생성
## No module named 'folium' 라는 에러가 뜬다면
## anaconda cmd 창에서 "pip install folium" 으로 설치
import folium
from folium.plugins import MiniMap

def make_map(df, region):
    # 해당 지역 시/도청 좌표 Search
    yp = addr_to_lat_lon(region, index)[0]
    xp = addr_to_lat_lon(region, index)[1]
    
    # 지도 생성하기
    m = folium.Map(location=[xp,yp],   # 뽑은 좌표
                   zoom_start=12)
    # 미니맵 추가하기
    minimap = MiniMap() 
    m.add_child(minimap)
    # 마커 추가하기
    for i in range(len(df.index)):
        folium.Marker(location=[df['위도'].iloc[i],df['경도'].iloc[i]],
                  popup=df['점수'].iloc[i],
                  tooltip=[df['식당명'].iloc[i], "\n점수", df['점수'].iloc[i]]
                  ).add_to(m)
    return m

In [91]:
region = str(input("검색을 원하는 지역의 \'시, 도\'를 입력해 주세요 : "))
page   = int(input("검색 범위(페이지)를 입력해주세요. : "))
starpoint = input("최대점수를 입력해 주세요(4.5 만점) : ")

driver = webdriver.Chrome("chromedriver")
#### time.sleep() 을 주기적으로 넣어 서버 측에 부하를 막고, 사람으로 인식하도록 처리한다
time.sleep(1)
driver.get('https://map.kakao.com/')
time.sleep(2)

#### 입력 받은 region에 ' 식당\n'이라는 string 을 추가해 날린다.
#### 마지막에 '\n'을 추가한 이유는 'enter'와 같은 기능을 한다고 보면 된다(검색어 입력)
driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]').send_keys('%s 식당\n' % region)
time.sleep(2)

#### 검색 완료 후, '정확도 순'에 맞춰 식당 데이터들을 정렬한다.
driver.find_element(By.XPATH, '//*[@id="info.search.place.sort"]/li[1]/a').send_keys(Keys.ENTER)
time.sleep(2)

n_list = []   # 식당 이름 리스트
s_list = []   # 식당 별점 리스트
a_list = []   # 식당 주소 리스트
px_list = []  # 경도 리스트
py_list = []  # 위도 리스트
ka_dict = {}  # 별점에 따른 가게를 담을 딕셔너리

검색을 원하는 지역의 '시, 도'를 입력해 주세요 : 경기도
검색 범위(페이지)를 입력해주세요. : 1
최대점수를 입력해 주세요(4.5 만점) : 3.5


In [93]:
source = driver.page_source
parsed_source = bs(source, 'html.parser')

names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')        # 가게명
names_span_list = parsed_source.find_all('span', class_ = 'screen_out')

In [96]:
names_span_list

[<span class="screen_out">더보기 메뉴</span>,
 <span class="screen_out" data-id="screenOutName">동동국수집</span>,
 <span class="screen_out">별점</span>,
 <span class="screen_out" data-id="screenOutName">진미통닭</span>,
 <span class="screen_out">별점</span>,
 <span class="screen_out" data-id="screenOutName">김네집</span>,
 <span class="screen_out">별점</span>,
 <span class="screen_out" data-id="screenOutName">포폴로피자 PIZZERIA DEL POPOLO</span>,
 <span class="screen_out">별점</span>,
 <span class="screen_out" data-id="screenOutName">나랏님이천쌀밥 본관</span>,
 <span class="screen_out">별점</span>,
 <span class="screen_out" data-id="screenOutName">허기숙할머니 원조오뎅식당 의정부본점</span>,
 <span class="screen_out">별점</span>,
 <span class="screen_out" data-id="screenOutName">북한강막국수닭갈비</span>,
 <span class="screen_out">별점</span>,
 <span class="screen_out" data-id="screenOutName">미가훠궈양고기</span>,
 <span class="screen_out">별점</span>,
 <span class="screen_out" data-id="screenOutName">설악막국수춘천닭갈비</span>,
 <span class="screen_out">별점</span>,
 <s

In [97]:
div_scores = parsed_source.find_all('em', class_ = "num")      # 점수
#del div_scores[-1]
for i in range(len(div_scores)):
    scores = div_scores[i].text
    s_list.append(scores)

In [99]:
div_scores

[<em class="num" data-id="scoreNum" title="3.6점">3.6</em>,
 <em class="num" data-id="scoreNum" title="3.0점">3.0</em>,
 <em class="num" data-id="scoreNum" title="3.6점">3.6</em>,
 <em class="num" data-id="scoreNum"></em>,
 <em class="num" data-id="scoreNum" title="2.0점">2.0</em>,
 <em class="num" data-id="scoreNum" title="3.4점">3.4</em>,
 <em class="num" data-id="scoreNum" title="3.9점">3.9</em>,
 <em class="num" data-id="scoreNum" title="4.0점">4.0</em>,
 <em class="num" data-id="scoreNum" title="4.1점">4.1</em>,
 <em class="num" data-id="scoreNum" title="2.4점">2.4</em>,
 <em class="num" data-id="scoreNum" title="2.4점">2.4</em>,
 <em class="num" data-id="scoreNum" title="3.1점">3.1</em>,
 <em class="num" data-id="scoreNum" title="3.9점">3.9</em>,
 <em class="num" data-id="scoreNum" title="3.4점">3.4</em>,
 <em class="num" data-id="scoreNum" title="3.7점">3.7</em>,
 <em class="num" data-id="reviewCount">0</em>]

In [92]:
source = driver.page_source
parsed_source = bs(source, 'html.parser')

names_div_list = parsed_source.find_all("li",class_ = 'PlaceItem clickArea')        # 가게명
names_span_list = parsed_source.find_all('span', class_ = 'screen_out')
span_names = names_span_list[1:31:2]
for i in range(len(span_names)):
    names = span_names[i].text
    n_list.append(names)


div_scores = parsed_source.find_all('em', class_ = "num")                           # 점수
del div_scores[-1]
for i in range(len(div_scores)):
    scores = div_scores[i].text
    s_list.append(scores)


div_address = parsed_source.find_all('div', class_ = "addr")                        # 주소
for i in range(len(div_address)):
    address = div_address[i].text.replace('\n','').split('(지번)')[0]
    try:
        px_list.append(addr_to_lat_lon(address, i)[0])
        py_list.append(addr_to_lat_lon(address, i)[1])
    except:
        pass
    a_list.append(address)

driver.find_element(By.XPATH,'//*[@id="info.search.page.no%s"]' % (j+1)).send_keys(Keys.ENTER)        # 페이지 넘기기
time.sleep(2)

0 https://dapi.kakao.com/v2/local/search/address.json?query=경기 남양주시 조안면 북한강로 914
0 https://dapi.kakao.com/v2/local/search/address.json?query=경기 남양주시 조안면 북한강로 914
1 https://dapi.kakao.com/v2/local/search/address.json?query=경기 파주시 지목로 114
1 https://dapi.kakao.com/v2/local/search/address.json?query=경기 파주시 지목로 114
2 https://dapi.kakao.com/v2/local/search/address.json?query=경기 용인시 수지구 이종무로 157
2 https://dapi.kakao.com/v2/local/search/address.json?query=경기 용인시 수지구 이종무로 157
3 https://dapi.kakao.com/v2/local/search/address.json?query=경기 파주시 광탄면 기산로 329 2층
3 https://dapi.kakao.com/v2/local/search/address.json?query=경기 파주시 광탄면 기산로 329 2층
4 https://dapi.kakao.com/v2/local/search/address.json?query=경기 수원시 팔달구 장다리로 282
4 https://dapi.kakao.com/v2/local/search/address.json?query=경기 수원시 팔달구 장다리로 282
5 https://dapi.kakao.com/v2/local/search/address.json?query=경기 수원시 팔달구 중부대로223번길 41
5 https://dapi.kakao.com/v2/local/search/address.json?query=경기 수원시 팔달구 중부대로223번길 41
6 https://dapi.kakao.com/v2/local/se